# Zweiter Korrekturschritt: Rahmenfehlstellung

Für ein ideales Nadir-Bild müssen die Bonitätsrahmen parallel zum Boden ausgerichtet werden. Außerdem muss das Tablet (für die Aufnahme) ebenfalls parallel zum Boden und zentriert über den Rahmen gehalten werden. Bei Abweichung kann es dazu kommen, dass der Bonitätrahmen im Nadir-Bilder nicht mehr (annähernd) quadratisch zu sehen ist, sondern aufgrund der Rahmenfehlstellung andere Formen (Rechteck, Trapez, ...) annimmt. Durch die unterschiedlichen Rahmen-Formen in den Bildern wird ein Vergleich der einzelnen Nadir-Bilder schwierig, weswegen im Folgenden eine perspektivische Transformation als Korrektur der Rahmenfehlstellung durchgeführt wird.

![Rahmenfehlstellung](media/frame.png)
<center>Rahmenfehlstellung</center>

Die perspketivische Transformation gelingt aufgrund von Detektions-Markern, welche an den Ecken des Bonitätsrahmens angebracht werden. Die Marker werden in den Nadir-Bildern detektiert und ihre Bildkoordinaten (Lage im Bild) werden gespeichert. Im Anschluss werden die Bildkoordinaten als Transformationspunkte für die perspektivische Transformation angewendet. Neben der reinen Transformation werden außerdem Rotation und Zuschnitt im selben Prozess mit ausgeführt. 

![Perspektivische Transformation](media/geom_transform.png)
<center>Perspektivische Transformation</center>


In [ ]:
# Zuweisung der Marker zu den Bildvierteln (links oben, links unten, rechts oben, rechts unten)
def check_position(coords):
    if coords[0] <= mid[0]:
        left_right = 0 # left
    else:
        left_right = 1 # right
        
    if coords[1] <= mid[1]:
        up_down = 0 # up
    else:
        up_down = 1 # down

    if left_right == 0 and up_down == 0:
        position = 0 # upper left
    elif left_right == 0 and up_down == 1:
        position = 1 # lower left
    elif left_right == 1 and up_down == 0:
        position = 2 # upper right
    elif left_right == 1 and up_down == 1:
        position = 3 # lower right
    return position

In [ ]:
# Sortiert die vier Koordinatenpaare eines jeden Markers und gibt eine sortierte (Lage) Liste aus
def get_corners(position):
    position = str(position)
    position = position.replace("[", "")
    position = position.replace("]", "")
    position = position.replace("'", "")
    position = position.split(",")

    corner1 = (float(position[0]), float(position[1]))
    corner2 = (float(position[2]), float(position[3]))
    corner3 = (float(position[4]), float(position[5]))
    corner4 = (float(position[6]), float(position[7]))

    corners_list = corner1, corner2, corner3, corner4
    #print(corners_list)

    corners_list = sorted(corners_list , key=lambda k: [k[0], k[1]])
    left = corners_list[0], corners_list[1]
    left = sorted(left , key=lambda k: [k[1], k[0]])
    upper_left_corner = left[0]
    lower_left_corner = left[1]
    right = corners_list[2], corners_list[3]
    right = sorted(right , key=lambda k: [k[1], k[0]])
    upper_right_corner = right[0]
    lower_right_corner = right[1]

    position_corner_list = upper_left_corner, lower_left_corner, upper_right_corner, lower_right_corner
    return position_corner_list

In [ ]:
import cv2, glob, os
from cv2 import aruco
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

output1_path = r"./output1/" # Pfad der primär (Verzeichnung) korrigierten Bildern
output2_path = r"./output2/" # Pfad der den sekundär (Rahmenfehlstellung) korrigierten Bildern
failed_path = r"./failed/" # Pfad der Bilder, bei denen die sek. Korrektur nicht automatisiert durchgeführt werden konnte
aruco_id = 0 # ID der zu detektierenden ArUco-Marker

nadir_images = glob.glob(output1_path + "*.jpg")
f = IntProgress(min=0, max=len(nadir_images), description='Korrektur:',bar_style='info')
display(f)

# Einlesen sämtlicher JPGs aus dem Ordner der verzeichnungsfreien Bilder
for image in nadir_images:
    print("\nNadir-Bild: " + image[10:])
    img = cv2.imread(image)

    height = img.shape[0] # Höhe
    width = img.shape[1] # Breite
    mid = (width/2, height/2)

    # Umwandlung in Graustufe
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Festlegen der ArUco-Parameter (4x4-Dict)
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_1000)
    parameters =  aruco.DetectorParameters_create()
    parameters.minDistanceToBorder = 0
    
    # Aufarbeiten/Prozessieren der Nadir-Bilder anhand von 9 binary threshholds und 2 adaptive thresholds
    ret, thresh100 = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    ret, thresh150 = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    ret, thresh200 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    ret, thresh210 = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY)
    ret, thresh220 = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
    ret, thresh230 = cv2.threshold(gray, 230, 255, cv2.THRESH_BINARY)
    ret, thresh240 = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    ret, thresh245 = cv2.threshold(gray, 245, 255, cv2.THRESH_BINARY)
    ret, thresh250 = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)
    adaptive_threshold1 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 1)
    adaptive_threshold11 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)

    thresh_list = gray, thresh100, thresh150, thresh200, thresh210, thresh220, thresh230, thresh240, thresh245, thresh250, adaptive_threshold1, adaptive_threshold11

    corners_list = []
    ids_list = []
    
    # Maker-Detektion für jede der 12 Aufarbeitungen (9 binary threshs, 2 adaptive threshs und normale Graustufe)
    for thresh in thresh_list:
        corners, ids, rejectedImgPoints = aruco.detectMarkers(thresh, aruco_dict, parameters=parameters)
        frame_markers = aruco.drawDetectedMarkers(img.copy(), corners, ids)
        
        if len(corners) != 0:
            corners = list(corners)
            corners_list.append(corners)
            ids_list.append(ids)
    
    # Herunterbrechen der detektierten Marker (Bildkoordinaten) ins Listenformat
    corners_list = str(corners_list)
    corners_list = corners_list.replace("\n", "")
    corners_list = corners_list.replace(" ", "")
    corners_list = corners_list.replace("array([", "")
    corners_list = corners_list.replace(",dtype=float32)", "")
    corners_list = corners_list.replace("[]", "")
    corners = corners_list.split("]],")

    # Herunterbrechen der detektierten ArUco-Ids ins Listenformat
    ids_list = str(ids_list)
    ids_list = ids_list.replace("\n", "")
    ids_list = ids_list.replace(" ", "")
    ids_list = ids_list.replace("array([", "")
    ids_list = ids_list.replace(",dtype=int32)", "")
    ids_list = ids_list.replace("[]", "")
    ids_list = ids_list.replace("[", "")
    ids_list = ids_list.replace("]", "")
    ids = ids_list.split(",")

    #print(corners)
    wrong_aruco_index_list = []

    # Kontrolle ob Marker detektiert wurden, deren ID nicht der vorher definierten entspricht (Fehldetektion)
    if len(corners) > 1 and len(ids) > 1:
        for zahl, marker_id in enumerate(ids):
            if int(marker_id) != int(aruco_id):
                ids = list(ids)
                index = ids.index(marker_id)
                wrong_aruco_index_list.append(index)

        # Bereinigung der Fehldetektionen
        for wrong_id_index in sorted(wrong_aruco_index_list, reverse=True):
            ids.pop(wrong_id_index)
            corners.pop(wrong_id_index)

    position_list = []
    upper_left = []
    upper_right = []
    lower_left = []
    lower_right = []
    
    # Liste mit detektierten Markern (Bildkoordinaten) wird durchlaufen und jeder Marker wird einem Bildviertel zugeteilt
    for corner in corners:
        if len(corner) != 0:
            cor = str(corner)
            cor = cor.replace("[", "")
            cor = cor.replace("]", "")
            cor = cor.split(",")
            #print(cor)
            coords = (float(cor[0]), float(cor[1]))
            #print(coords)
            
            # Zuteilung der Marker in ein Bildviertel
            position = check_position(coords)

            # Speichern der Marker in den entsprechenden Listen
            if position == 0:
                upper_left.append(corner)
            elif position == 1:
                lower_left.append(corner)
            elif position == 2:
                upper_right.append(corner)
            elif position == 3:
                lower_right.append(corner)

            # Kontrollliste zur Überprüfung ob Marker für alle vier Bildviertel vorliegen
            if position not in position_list:
                position_list.append(position)

    # Sollten für alle vier Bildviertel Marker vorliegen, wird der Median der Bildkoordinaten ausgewählt
    if len(position_list) == 4:
        corner_index = round((len(upper_left))/2)
        upper_left = upper_left[corner_index]
        corner_index = round((len(lower_left))/2)
        lower_left = lower_left[corner_index]
        corner_index = round((len(upper_right))/2)
        upper_right = upper_right[corner_index]
        corner_index = round((len(lower_right))/2)
        lower_right = lower_right[corner_index]

        # Wenn vier Median-Koordinaten vorliegen werden die Koordinatenpaare eines jeden Markers nach Ausrichtung sortiert
        if len(upper_left) != 0 and len(upper_right) != 0 and len(lower_left) != 0 and len(lower_right) != 0:
            point1 = get_corners(upper_left)
            point1 = point1[3]
            point2 = get_corners(lower_left)
            point2 = point2[2]
            point3 = get_corners(upper_right)
            point3 = point3[1]
            point4 = get_corners(lower_right)
            point4 = point4[0]

            rows,cols,ch = img.shape

            pts1 = np.float32([point1,point2,point3,point4])
            
            #Bildgroesse festgelegt auf 1000x1000 Pixel
            length_warp = 1000

            # Größe des Bildes, welches ausgegeben wird
            pts2 = np.float32([[0,0],[length_warp,0],[0,length_warp],[length_warp,length_warp]])
            
            # Erzeugen der Transformationsmatrix anhand Ist-Koordinaten (Marker) und Soll-Koordinaten
            perspective = cv2.getPerspectiveTransform(pts1,pts2)
            
            # Perspektivische Transformation
            dst = cv2.warpPerspective(img,perspective,(length_warp, length_warp))

            if not os.path.isdir(output2_path):
                os.mkdir(output2_path)

            cv2.imwrite(output2_path + image[10:], dst)
            print("Korrigiert. Ausgabepfad: " + output2_path + image[10:])
    else:
        if not os.path.isdir(failed_path):
            os.mkdir(failed_path)

        cv2.imwrite(failed_path + image[10:], img)
        print("Korrektur fehlgeschlagen: Es konnten keine 4 ArUco-Marker gefunden werden.\nAusgabepfad: " + failed_path + image[10:])
    
    f.value += 1